In [1]:
%%javascript
var width = window.innerWidth || document.documentElement.clientWidth || document.body.clientWidth;
var height = window.innerHeight || document.documentElement.clientHeight || document.body.clientHeight;

IPython.notebook.kernel.execute("windowSize = (" + width + "," + height + ")");
// suitable for small screens
nbpresent.mode.tree.set(
    ["app", "theme-manager", "themes", "my-theme"], 
    {
    palette: {
        "blue": { id: "blue", rgb: [0, 153, 204] },
        "black": { id: "black", rgb: [0, 0, 0] },
        "white": { id: "white", rgb: [255, 255, 255] },
        "red": { id: "red", rgb: [240, 32, 32] },
        "gray": { id: "gray", rgb: [128, 128, 128] },
    },
    backgrounds: {
        "my-background": {
            "background-color": "white"
        }
    },
    "text-base": {
        "font-family": "Georgia",
        "font-size": 2.5
    },
    rules: {
        h1: {
            "font-size": 5.5,
            color: "blue",
            "text-align": "center"
        },
        h2: {
            "font-size": 3,
            color: "blue",
            "text-align": "center"
        },
        h3: {
            "font-size": 3,
            color: "black",
        },
        "ul li": {
            "font-size": 2.5,
            color: "black"
        },
        "ul li ul li": {
            "font-size": 2.0,
            color: "black"
        },
        "code": {
            "font-size": 1.6,
        },
        "pre": {
            "font-size": 1.6,
        }
    }
});

<IPython.core.display.Javascript object>

# Genome Rearrangements - Continued

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/breakpoint.jpg" width="700px" class="centerImg">

<p style="text-align: right; clear: right;">1</p>

# Lessons from last time

1. With each reversal, one can remove at most 2 breakpoints
2. If there is any *decreasing* strip there exists a reversal that will remove at least one breakpoint
3. If breakpoints remain and there is no *decreasing* strip one can be created by reserving *any* remaining strip

$$
\begin{aligned}
& \overrightarrow{\color{blue}{0}, 1, 2},\color{red}{|}\:\underleftarrow{\overrightarrow{5, 6, 7},}\color{red}{|}\:\overrightarrow{3, 4}, \color{red}{|}\: \overrightarrow{8,\color{blue}{9}}\qquad & b(p) = 3\qquad & \rho(3, 5)\\
& \overrightarrow{\color{blue}{0}, 1, 2},\color{red}{|}\:\overleftarrow{7, 6, 5},\color{red}{|}\:\overrightarrow{3, 4}, \color{red}{|}\: \overrightarrow{8,\color{blue}{9}}\qquad & b(p) = 3\qquad & \rho(6, 7)\\
& \overrightarrow{\color{blue}{0}, 1, 2},\color{red}{|}\:\overleftarrow{7, 6, 5, 4, 3}, \color{red}{|}\: \overrightarrow{8,\color{blue}{9}}\qquad & b(p) = 2\qquad & \rho(3, 7)\\
& \overrightarrow{\color{blue}{0}, 1, 2, 3, 4, 5, 6, 7, 8,\color{blue}{9}}\qquad & b(p) = 0\qquad  & Done!\\
\end{aligned}
$$

An optimal algorithm would remove 2 breakpoints at every step. The last reversal always removes 2 breakpoints, thus if the number of breakpoints is odd, even the optimal algorithm must make at least one reersal that removes only 1 breakpoint. 

<p style="text-align: right; clear: right;">2</p>

# An Improved Breakpoint Reversal Sort

### ImprovedBreakpointReversalSort(π)
```
1. while b(π) > 0
2.     if π has a decreasing strip
3.         Among all possible reversals, choose reversal ρ that minimizes b(π • ρ)
4.     else
5.         Choose a reversal ρ that flips an increasing strip in π
6.     π ← π • ρ
7. output π
8. return
```

<p style="text-align: right; clear: left;">3</p>

# Improved Breakpoint Reversal Sort in Python

In [1]:
def hasBreakpoints(seq):
    """ returns True if sequnces in not strictly increasing by 1 """
    for i in xrange(1, len(seq)):
        if (seq[i] != seq[i-1] + 1):
            return True
    return False

def getStrips(seq):
    """ find contained intervals where sequence is ordered, and return intervals
    in as lists, increasing and decreasing. Single elements are considered
    decreasing. "Contained" excludes the first and last interval. """
    deltas = [seq[i+1] - seq[i] for i in xrange(len(seq)-1)]
    increasing = list()
    decreasing = list()
    start = 0
    for i, diff in enumerate(deltas):
        if (abs(diff) == 1) and (diff == deltas[start]):
            continue
        if (start > 0):
            if deltas[start] == 1:
                increasing.append((start, i+1))
            else:
                decreasing.append((start, i+1))
        start = i+1
    return increasing, decreasing

def pickReversal(seq, strips):
    """ test each decreasing interval to see if it leads to a reversal that
    removes two breakpoints, otherwise, return a reversal that removes only one """
    for i, j in strips:
        k = seq.index(seq[j-1]-1)
        if (seq[k+1] + 1 == seq[j]):
            # removes 2 breakpoints
            return 2, (min(k+1, j), max(k+1, j))
    # In the worst case we remove only one, but avoid the length "1" strips
    for i, j in strips:
        k = seq.index(seq[j-1]-1)
        if (j - i > 1):
            break
    return 1, (min(k+1, j), max(k+1, j))

def doReversal(seq,(i,j)):
    return seq[:i] + [element for element in reversed(seq[i:j])] + seq[j:]

In [2]:
def improvedBreakpointReversalSort(seq, verbose=True):
    seq = [0] + seq + [max(seq)+1]                             # Extend sequence
    N = 0
    while hasBreakpoints(seq):
        increasing, decreasing = getStrips(seq)
        if len(decreasing) > 0:                                # pick a reversal that removes a decreasing strip
            removed, reversal = pickReversal(seq, decreasing)
        else:
            removed, reversal = 0, increasing[0]               # No breakpoints can be removed
        if verbose:
            print "Strips:", increasing, decreasing
            print "%d: %s  rho%s" % (removed, seq, reversal)
            raw_input("Press Enter:")
        seq = doReversal(seq,reversal)
        N += 1
    if verbose:
        print seq, "Sorted"
    return N

# Also try: [1,9,3,4,7,8,2,6,5]
print improvedBreakpointReversalSort([3,4,1,2,5,6,7,10,9,8], verbose=True)

Strips: [(1, 3), (3, 5), (5, 8)] [(8, 11)]
2: [0, 3, 4, 1, 2, 5, 6, 7, 10, 9, 8, 11]  rho(8, 11)
Press Enter:
Strips: [(1, 3), (3, 5)] []
0: [0, 3, 4, 1, 2, 5, 6, 7, 8, 9, 10, 11]  rho(1, 3)
Press Enter:
Strips: [(3, 5)] [(1, 3)]
1: [0, 4, 3, 1, 2, 5, 6, 7, 8, 9, 10, 11]  rho(3, 5)
Press Enter:
Strips: [] [(1, 5)]
2: [0, 4, 3, 2, 1, 5, 6, 7, 8, 9, 10, 11]  rho(1, 5)
Press Enter:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] Sorted
4


<p style="text-align: right; clear: right;">4</p>

# Performance
<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/CanWeDoBetter.png" width="250px" style="float: right; clear: right; margin-right: 40px;">
* *ImprovedBreakPointReversalSort* is a greedy algorithm with a performance guarantee of no worse than 4 when compared to an optimal algorithm
    - It eliminates at least one breakpoint in every two steps (flip an increasing then remove 1)
    - That's at most: $2 b(\Pi)$ steps
    - An optimal algorithm could *at most* remove 2 breakpoints in every step, thus requiring $\frac{b(\Pi)}{2}$ steps
    - The approximation ratio is:
$$\frac{\mathcal{A}(\Pi)}{OPT(\Pi)} = \frac{2 b(\Pi)}{\frac{b(\Pi)}{2}} = 4$$

* But there is a solution with far fewer flips

<p style="text-align: right; clear: right;">5</p>

# A Better Approximation Ratio

* If there is a decreasing strip, the next reversal reduces b(π) by at least one.
* The only bad case is when there is no decreasing strip.<br>Then we do a reversal that does not reduce b(π).
* If we always choose a reversal reducing b(π) and, *at the same time, select a permutation such that the result has at least one decreasing strip*, the bad case would never occur.
* If all possible reversals that reduce b(π) create a permutation without decreasing strips, then there exists a reversal that reduces b(π) by 2 (Proof not given)! 
* When the algorithm creates a permutation without a decreasing strip, the previous reversal must have reduced b(π) by two.
* At most b(π) reversals are needed.
* The improved Approximation ratio:

$$\frac{\mathcal{A_{new}}(\Pi)}{OPT(\Pi)} = \frac{b(\Pi)}{\frac{b(\Pi)}{2}} = 2$$

<p style="text-align: right; clear: left;">6</p>

# Comparing Greedy Algorithms

### SimpleReversalSort

* Attempts to extend the prefix(π) at each step
* Approximation ratio $\frac{n-1}{b(\Pi)/2}$ steps

### ImprovedBreakpointReversalSort

* Attempts to reduce the numbe of breakpoints at each step
* Approximation ratio $\frac{b(\Pi)}{b(\Pi)/2} = 2$ steps

<img src="http://csbio.unc.edu/mcmillan/Comp555S18/Media/ManMouseX.png" width="300px" class="centerImg">

<p style="text-align: right; clear: right;">7</p>



<br><br><br><br><br><br>
# Problem Set Time!
<br><br><br><br><br><br>
<p style="text-align: right; clear: right;">8</p>